In [35]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV


In [36]:
data = pd.read_csv("../data/model_ready.csv")

In [37]:
X = data[["Timestamp", "Latitude", "Longitude"]]
y = data[["Magnitude", "Depth"]]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(18727, 3) (4682, 3) (18727, 2) (4682, 2)


In [38]:
def create_model(neurons, activation, optimizer, loss):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_shape=(3,)))
    model.add(Dense(neurons, activation))
    model.add(Dense(2, activation='softmax'))


    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

    return model

In [41]:
model = KerasRegressor(model=create_model, verbose=0)

param_grid = {
    'model__neurons': [16, 64],
    'model__activation': ['sigmoid', 'relu'],
    'model__optimizer': ['SGD', 'Adadelta'],
    'model__loss': ['mse', 'mae'],   # better regression losses than square_hinge
    'batch_size': [10],
    'epochs': [10]
}


In [42]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']


for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

c:\Users\pc\End to End project\Earthquake Prediction\.venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best: -68.223668 using {'batch_size': 10, 'epochs': 10, 'model__activation': 'sigmoid', 'model__loss': 'mae', 'model__neurons': 64, 'model__optimizer': 'SGD'}
-97.935523 (3.807625) with: {'batch_size': 10, 'epochs': 10, 'model__activation': 'sigmoid', 'model__loss': 'mse', 'model__neurons': 16, 'model__optimizer': 'SGD'}
-87.905362 (6.298605) with: {'batch_size': 10, 'epochs': 10, 'model__activation': 'sigmoid', 'model__loss': 'mse', 'model__neurons': 16, 'model__optimizer': 'Adadelta'}
-97.935670 (3.807640) with: {'batch_size': 10, 'epochs': 10, 'model__activation': 'sigmoid', 'model__loss': 'mse', 'model__neurons': 64, 'model__optimizer': 'SGD'}
-93.960750 (4.489568) with: {'batch_size': 10, 'epochs': 10, 'model__activation': 'sigmoid', 'model__loss': 'mse', 'model__neurons': 64, 'model__optimizer': 'Adadelta'}
-70.229312 (3.221998) with: {'batch_size': 10, 'epochs': 10, 'model__activation': 'sigmoid', 'model__loss': 'mae', 'model__neurons': 16, 'model__optimizer': 'SGD'}
-81.165253 

In [43]:
print("Shape:", y_train.shape)
print(y_train.head())
print("Describe:\n", y_train.describe())


Shape: (18727, 2)
       Magnitude  Depth
16953        6.2   10.0
15800        5.6  217.3
9014         6.1   10.0
15516        5.5   10.0
17837        6.3   21.0
Describe:
           Magnitude         Depth
count  18727.000000  18727.000000
mean       5.882290     70.608168
std        0.421031    122.497372
min        5.500000     -1.100000
25%        5.600000     14.595000
50%        5.700000     33.000000
75%        6.000000     54.000000
max        9.100000    691.600000
